### It works!

In [ ]:
import requests
import urllib.parse
import datetime
import json
from credentials import credentials

# Get token (abvailable for 1 hour)
token_response = requests.post(
    'https://api.amazon.com/auth/o2/token',
    data={
        'grant_type': 'refresh_token',
        'refresh_token': credentials['refresh_token'],
        'client_id': credentials['lwa_app_id'],
        'client_secret': credentials['lwa_client_secret'],
    }
)

access_token = token_response.json()['access_token']

endpoint = 'https://sellingpartnerapi-eu.amazon.com'
marketplace_id = 'A1F83G8C2ARO7P'

# Download FBA inventory report
request_params = {
    'details': 'true',
    'granularityType': 'Marketplace',
    'granularityId': marketplace_id,
    'marketplaceIds': marketplace_id,
    'startDateTime': (datetime.datetime.now() - datetime.timedelta(days=1)).isoformat(),
}

inventory_report = requests.get(
    endpoint
    + '/fba/inventory/v1/summaries'
    + '?'
    + urllib.parse.urlencode(request_params),
    headers={
        'x-amz-access-token': access_token,
    },
)

print(json.dumps(inventory_report.json(), indent=2))

{
  "payload": {
    "granularity": {
      "granularityType": "Marketplace",
      "granularityId": "A1F83G8C2ARO7P"
    },
    "inventorySummaries": [
      {
        "asin": "B0BPHWJT4Q",
        "fnSku": "B0BPHWJT4Q",
        "sellerSku": "HA1138",
        "condition": "NewItem",
        "inventoryDetails": {
          "fulfillableQuantity": 57,
          "inboundWorkingQuantity": 0,
          "inboundShippedQuantity": 0,
          "inboundReceivingQuantity": 0,
          "reservedQuantity": {
            "totalReservedQuantity": 0,
            "pendingCustomerOrderQuantity": 0,
            "pendingTransshipmentQuantity": 0,
            "fcProcessingQuantity": 0
          },
          "researchingQuantity": {
            "totalResearchingQuantity": 0,
            "researchingQuantityBreakdown": [
              {
                "name": "researchingQuantityInShortTerm",
                "quantity": 0
              },
              {
                "name": "researchingQuantityInMidTe

### Inventory and Sales

In [6]:
import json
import time
import csv
from datetime import datetime, timedelta
import pytz

import pandas as pd
import requests
import seaborn as sns
from sp_api.api import Reports, Sales
from sp_api.base import Marketplaces, ReportType, ProcessingStatus, Granularity

from typing import List
import requests
import pandas as pd

if __name__ == '__main__':

    #report_type = ReportType.GET_FBA_MYI_ALL_INVENTORY_DATA
    #res = Reports(credentials=credentials, marketplace=Marketplaces.UK)
    #data = res.create_report(reportType=report_type)
    #report = data.payload
    print('Got the report id')

    report_id = '234820019915'
    res = Reports(credentials=credentials, marketplace=Marketplaces.UK)
    data = res.get_report(report_id)
    while data.payload.get('processingStatus') not in [ProcessingStatus.DONE, ProcessingStatus.FATAL, ProcessingStatus.CANCELLED]:
        print(data.payload)
        print('Waiting for report to be processed...')
        time.sleep(2)
        data = res.get_report(report_id)
    
    if data.payload.get('processingStatus') in [ProcessingStatus.FATAL, ProcessingStatus.CANCELLED]:
        print('Report processing failed')
        report_data = data.payload

    else:
        print('Successfully downloaded report data')
        report_data = res.get_report_document(data.payload['reportDocumentId'])
    
    report_url = report_data.payload['url']

    res = requests.get(report_url)

    decoded_content = res.content.decode('cp1252')
    reader = csv.DictReader(decoded_content.splitlines(), delimiter='\t')

    data_list = list(reader)
    inventory = pd.DataFrame(data_list)
    
    timezone = pytz.timezone('Europe/London')
    end_date = datetime.now(timezone)
    start_date = end_date - timedelta(days=30)
    start_date_str = start_date.isoformat()
    end_date_str = end_date.isoformat()

    asins = list(inventory['asin'])
    marketplaces = dict(UK=Marketplaces.UK)
    data = []
    for asin in asins:
        for country, marketplace_id in marketplaces.items():
            sales = Sales(credentials=credentials, marketplace=marketplace_id)
            res = sales.get_order_metrics(
                interval=(start_date_str, end_date_str),
                granularity=Granularity.TOTAL,
                asin=asin)
            metrics = res.payload[0]
            data.append({'asin': asin,
                         'unit_count': metrics['unitCount'],
                         #'order_item_count': metrics['orderItemCount'],
                         #'order_count': metrics['orderCount'],
                         #'country': country,
                         })
            print(f'Got the sales for asin {asin}')
            time.sleep(2)
    print('Succesfully got the sales data')
    sales = pd.DataFrame(data)

    df = pd.merge(inventory, sales, on='asin', how='left')


    df.to_csv('reports/data.csv', index=False)


Got the report id
Successfully downloaded report data
Got the sales for asin B0CTS8FL6T
Got the sales for asin B0CQD1DT1P
Got the sales for asin B0CY33HRL6
Got the sales for asin B09RVJ59T7
Got the sales for asin B0C8GPYCR7
Got the sales for asin B09RFDFHYW
Got the sales for asin B0B5GY7NP4
Got the sales for asin B09RVJ7DDN
Got the sales for asin B09RVJ43MH
Got the sales for asin B09RGHWD5H
Got the sales for asin B0BPHWJT4Q
Got the sales for asin B09RVJNDPW
Got the sales for asin B0B8XTX3LB
Got the sales for asin B0BPHXJQ47
Got the sales for asin B0C8GPDTJH
Got the sales for asin B0C8GP67SH
Got the sales for asin B0CPTGQBJF
Got the sales for asin B0CPTCDTKK
Got the sales for asin B0CPTGVWSD
Got the sales for asin B0CPTFBJQK
Got the sales for asin B0CPTF8TXV
Got the sales for asin B0CPTDVJ91
Got the sales for asin B0CPTG3GG1
Got the sales for asin B0CPTFR9BP
Got the sales for asin B0CV656QCW
Got the sales for asin B0CV654W5B
Got the sales for asin B0CV5Z613L
Got the sales for asin B0CV6

In [15]:
import json
import time
import csv
from datetime import datetime, timedelta
import pytz

import pandas as pd
import requests
import seaborn as sns
from sp_api.api import Reports, Sales
from sp_api.base import Marketplaces, ReportType, ProcessingStatus, Granularity

from typing import List
import requests
import pandas as pd

from credentials import credentials

report_id = '234820019915'
res = Reports(credentials=credentials, marketplace=Marketplaces.UK)
data = res.get_report(report_id)
while data.payload.get('processingStatus') not in [ProcessingStatus.DONE, ProcessingStatus.FATAL, ProcessingStatus.CANCELLED]:
    print(data.payload)
    print('Waiting for report to be processed...')
    time.sleep(2)
    data = res.get_report(report_id)

if data.payload.get('processingStatus') in [ProcessingStatus.FATAL, ProcessingStatus.CANCELLED]:
    print('Report processing failed')
    report_data = data.payload

else:
    print('Successfully downloaded report data')
    report_data = res.get_report_document(data.payload['reportDocumentId'])

report_url = report_data.payload['url']

res = requests.get(report_url)

decoded_content = res.content.decode('cp1252')
reader = csv.DictReader(decoded_content.splitlines(), delimiter='\t')

data_list = list(reader)
inventory = pd.DataFrame(data_list)

timezone = pytz.timezone('Europe/London')
end_date = datetime.now(timezone)
start_date = end_date - timedelta(days=30)
start_date_str = start_date.isoformat()
end_date_str = end_date.isoformat()

asins = list(inventory['asin'])
marketplaces = dict(UK=Marketplaces.UK)
data = []
for asin in asins:
    for country, marketplace_id in marketplaces.items():
        sales = Sales(credentials=credentials, marketplace=marketplace_id)
        res = sales.get_order_metrics(
            interval=(start_date_str, end_date_str),
            granularity=Granularity.TOTAL,
            asin=asin)
        metrics = res.payload[0]
        data.append({'asin': asin,
                        'unit_count': metrics['unitCount'],
                        #'order_item_count': metrics['orderItemCount'],
                        #'order_count': metrics['orderCount'],
                        #'country': country,
                        })
        print(f'Got the sales for asin {asin}')
        time.sleep(2)
print('Succesfully got the sales data')
sales = pd.DataFrame(data)

df = pd.merge(inventory, sales, on='asin', how='left')


🌟 Thank you for using python-amazon-sp-api! 🌟
This tool helps developers and businesses connect seamlessly with Amazon's vast marketplace,
enabling powerful automations and data management.
If you appreciate this project and find it useful, please consider supporting its continued development:
 - 🙌 GitHub Sponsors: https://github.com/sponsors/saleweaver
 - 🌐 BTC Address: bc1q6uqgczasmnvnc5upumarugw2mksnwneg0f65ws
 - 🌐 ETH Address: 0xf59534F7a7F5410DBCD0c779Ac3bB6503bd32Ae5

Your support helps keep the project alive and evolving, and is greatly appreciated!

To disable this donation message, set the ENV_DISABLE_DONATION_MSG=1 environment variable.
Successfully downloaded report data
🌟 Thank you for using python-amazon-sp-api! 🌟
This tool helps developers and businesses connect seamlessly with Amazon's vast marketplace,
enabling powerful automations and data management.
If you appreciate this project and find it useful, please consider supporting its continued development:
 - 🙌 GitHub Sp

In [16]:
df.head()

,sku,fnsku,asin,product-name,condition,your-price,mfn-listing-exists,mfn-fulfillable-quantity,afn-listing-exists,afn-warehouse-quantity,...,per-unit-volume,afn-inbound-working-quantity,afn-inbound-shipped-quantity,afn-inbound-receiving-quantity,afn-researching-quantity,afn-reserved-future-supply,afn-future-supply-buyable,afn-fulfillable-quantity-local,afn-fulfillable-quantity-remote,unit_count
0,HA1014,B0CTS8FL6T,B0CTS8FL6T,Sons Strengthening Conditioner For Men - DHT B...,New,14.99,No,,Yes,99,...,815.26,0,0,0,0,0,0,95,0,63
1,HA1018,B0CQD1DT1P,B0CQD1DT1P,"Sons Hair Complex Multi-Vitamin & Anagain, Hai...",New,29.99,No,,Yes,0,...,,0,0,0,0,0,0,0,0,0
2,HA1107,X001IJKQWH,B0CY33HRL6,"Sons Minoxidil 5% Cutaneous Solution, Hair Reg...",New,16.99,No,,Yes,5898,...,995.35,0,0,0,68,0,0,0,0,831
3,HA1107NB,B09RVJ59T7,B09RVJ59T7,"Sons Minoxidil 5% Cutaneous Solution, Hair Reg...",New,19.33,No,,Yes,7613,...,431.14,0,0,0,9,0,0,7242,0,934
4,HA1114,B0C8GPYCR7,B0C8GPYCR7,Sons Hair Growth Shampoo For Men - DHT Blockin...,New,14.99,No,,Yes,35,...,724.47,0,200,0,6,0,0,10,0,507


In [20]:
from product_groups import product_groups

# Function to determine product group based on the product name
def get_product_group(product_name):
    for group, products in product_groups.items():
        if product_name in products:
            return group
    return "Unknown"

# Add 'Product Group' column based on the product name
df['Product Group'] = df['product-name'].apply(get_product_group)

# Step 2: Select and rename the relevant columns
df_selected = df[['Product Group', 'product-name', 'sku', 'afn-fulfillable-quantity',
                  'afn-inbound-shipped-quantity', 'afn-inbound-receiving-quantity', 
                  'afn-inbound-working-quantity', 'unit_count']].copy()

df_selected.rename(columns={
    'product-name': 'Product',
    'sku': 'SKU',
    'afn-fulfillable-quantity': 'Available',
    'afn-inbound-shipped-quantity': 'Shipped',
    'unit_count': 'Units Sold T-30 Days'
}, inplace=True)

# Step 3: Convert necessary columns to numeric, coerce errors to NaN (if any)
df_selected['Units Sold T-30 Days'] = pd.to_numeric(df_selected['Units Sold T-30 Days'], errors='coerce')
df_selected['Available'] = pd.to_numeric(df_selected['Available'], errors='coerce')
df_selected['Shipped'] = pd.to_numeric(df_selected['Shipped'], errors='coerce')
df['afn-inbound-receiving-quantity'] = pd.to_numeric(df['afn-inbound-receiving-quantity'], errors='coerce')
df['afn-inbound-working-quantity'] = pd.to_numeric(df['afn-inbound-working-quantity'], errors='coerce')

# Step 4: Compute the 'Processing' column
df_selected['Processing'] = df['afn-inbound-receiving-quantity'] + df['afn-inbound-working-quantity']

# Step 5: Compute the 'Total Inventory' column
df_selected['Total Inventory'] = df_selected['Available'] + df_selected['Shipped'] + df_selected['Processing']

# Step 6: Compute the 'Weekly Run Rate' column
#df_selected['Weekly Run Rate'] = df_selected['Units Sold T-30 Days'] / 4

# Step 7: Compute the 'Weeks of Cover' column
#df_selected['Weeks of Cover'] = df_selected['Total Inventory'] / df_selected['Weekly Run Rate']

# Step 8: Compute the 'Recommendation' column
#def recommend_weeks_of_cover(weeks):
#    if math.isnan(weeks):
#        return 'Prepare pack off'
#    elif weeks < 7:
#        return 'Prepare pack off'
#    elif 7 <= weeks <= 16:
#        return 'Keep POP'
#    else:
#       return 'Hold pack offs'


#df_selected['Recommendation'] = df_selected['Weeks of Cover'].apply(recommend_weeks_of_cover)

# Display the resulting DataFrame
df_selected.drop(columns=['afn-inbound-receiving-quantity', 'afn-inbound-working-quantity'], inplace=True)
df_selected = df_selected[['Product Group',
                           'Product',
                           'SKU',
                           'Available',
                           'Shipped',
                           'Processing',
                           'Total Inventory',
                           'Units Sold T-30 Days']]



In [21]:
df_selected

,Product Group,Product,SKU,Available,Shipped,Processing,Total Inventory,Units Sold T-30 Days
0,Rest,Sons Strengthening Conditioner For Men - DHT B...,HA1014,95,0,0,95,63
1,Rest,"Sons Hair Complex Multi-Vitamin & Anagain, Hai...",HA1018,0,0,0,0,0
2,Minoxidil,"Sons Minoxidil 5% Cutaneous Solution, Hair Reg...",HA1107,0,0,0,0,831
3,Minoxidil,"Sons Minoxidil 5% Cutaneous Solution, Hair Reg...",HA1107NB,7242,0,0,7242,934
4,Rest,Sons Hair Growth Shampoo For Men - DHT Blockin...,HA1114,10,200,0,210,507
5,Rest,Sons Hair Growth Shampoo For Men - DHT Blockin...,HA1115,0,60,0,60,64
6,Rest,Sons Hair Growth Shampoo For Men - DHT Blockin...,HA1116,0,0,0,0,3
7,Bundles w/ Minoxidil,Sons Complete Hair Loss Treatment - Minoxidil ...,HA1123,1270,200,0,1470,245
8,Bundles w/ Minoxidil,Sons Complete Hair Loss Treatment - Minoxidil ...,HA1124,239,40,0,279,46
9,Rest,Sons Biotin 900ug Capsules - Hair Growth Vitam...,HA1137,212,0,0,212,83


### Extra stuff

In [13]:
import yaml

with open("environment.yml", "r") as f:
    env = yaml.safe_load(f)

requirements = []

for dep in env['dependencies']:
    if isinstance(dep, str):
        requirements.append(dep)
    elif isinstance(dep, dict) and dep.get('pip'):
        requirements.extend(dep['pip'])

with open("requirements.txt", "w") as f:
    for req in requirements:
        f.write(f"{req}\n")

In [14]:
import json
import time
import csv
from datetime import datetime, timedelta
import pytz

import pandas as pd
import requests
import seaborn as sns
from sp_api.api import Reports, Sales
from sp_api.base import Marketplaces, ReportType, ProcessingStatus, Granularity

from typing import List
import requests
import pandas as pd

if __name__ == '__main__':

    report_type = ReportType.GET_FBA_MYI_ALL_INVENTORY_DATA
    res = Reports(credentials=credentials, marketplace=Marketplaces.UK)
    data = res.create_report(reportType=report_type)
    report = data.payload
    print(report)
    report_id = report['reportId']


{'reportId': '234926019915'}


In [15]:
report_id

'234926019915'